# Slikar

This problem asks to check if some animals can reach a den, while the 2D terrain is being flooded,
and if so, in how many orthogonal moves.


## Tests

This function tests my program with the provided input and output files.

In [1]:
import glob
import subprocess
from timeit import default_timer as timer

def test():
    problem = 'slikar'
    limit = 1 # seconds per test
    for infile in sorted(glob.iglob(f'{problem}/*.in.*')):
        outfile = infile.replace('in', 'out')
        command = f'python {problem}.py < {infile} | diff -w - {outfile}'
        print('Running on', infile)
        start = timer()
        differences = subprocess.run(command, shell=True, text=True,
                                     stdout=subprocess.PIPE)
        end = timer()
        if differences.stdout:
            print(differences.stdout)
        elif end - start > limit:
            print('Too long:', end - start, 'seconds')

## Official solution

The main idea is to simulate the movement of the animals and the spread of the water.
We start with a 2D matrix of characters representing the initial state of the terrain.
After each step, the matrix shows all positions the animals could have reached by then, and all positions flooded.

Given the previous matrix, the matrix after the current step is the same, except that:

- Any field that is neither a rock (`X`) nor the den (`D`) and is adjacent to water (`*`), becomes water.
- Any empty field (`.`) not adjacent to water but adjacent to a position the animals can reach (`S`),
  becomes reachable.

If a position adjacent to the den is reachable in the previous matrix,
we can stop as the den is reached in this step.
If after this step no reachable positions remain, we can stop too: the water caught up with the animals.

With 2D grids, a common trick is to add 2 extra rows and columns (with rocks, in this case) around the grid,
so that we don't have to check for the edge of the terrain when looking at adjacent fields.
Only orthogonal adjacency is considered in this problem.

In [2]:
%%writefile slikar.py

def adjacent(row, col):
    """Return string of the orthogonally adjacent fields in the previous step."""
    return previous[row-1][col] + previous[row+1][col] + previous[row][col-1] + previous[row][col+1]

rows, columns = [int(string) for string in input().split()]

previous = [ list('X' * (columns + 2)) ]                # first row is rocks
for row in range(rows):
    previous.append(list('X' + input().rstrip() + 'X')) # rocks around the edges
previous.append(list('X' * (columns + 2)))              # last row is rocks

moves = 0
alive = True
reached = False

while not reached and alive:
    current = [row[:] for row in previous]              # copy the 2D grid
    for row in range(1, rows+1):
        for col in range(1, columns+1):
            fields = adjacent(row, col)
            # water floods adjacent empty and reachable fields
            if previous[row][col] in '.S' and '*' in fields:
                current[row][col] = '*'
            # animals reach adjacent empty fields
            elif previous[row][col] == '.' and 'S' in fields:
                current[row][col] = 'S'
            # animals reach the den if adjacent
            elif previous[row][col] == 'D' and 'S' in fields:
                reached = True
                current[row][col] = 'S'                 # keeps alive == True
    # check if the animals drowned (reachable positions were flooded)
    alive = any('S' in row for row in current)
    moves += 1
    previous = current

print(moves if reached else 'KAKTUS')

Overwriting slikar.py


Calling `rstrip` shouldn't be necessary, as `input` removes the final newline,
but for some reason the above test setup doesn't work without it.

The order of the first two if-statements is important: 
if both animals and water can reach an empty field in this step, the field becomes flooded and not reachable. 

In [3]:
test()

Running on slikar/slikar.in.1
Running on slikar/slikar.in.10
Running on slikar/slikar.in.2
Running on slikar/slikar.in.3
Running on slikar/slikar.in.4
Running on slikar/slikar.in.5
Running on slikar/slikar.in.6
Running on slikar/slikar.in.7
Running on slikar/slikar.in.8
Running on slikar/slikar.in.9
